![Podmanlogo](Pictures/podman-logo.png)

# Managing pods

All this time we've been working with simple containers, but Podman received its name because it's also a pod manager. But, what are a pods? Pods are a group of one or more containers sharing the same network, pid and ipc namespaces. If two containers are in the same pod they can reach each other by using localhost or 127.0.0.1 adress.

Moreover, pods are the smaller atomic unit that kubernetes clusters can manage. Running pods is super useful to run local tests without the need of deploying them on a kubernetes cluster.

Let's see how to manage pods with Podman.

First we are going to create a pod, for the moment it'll be empty.

In [3]:
!podman pod create --name my-pod

f0504fbc7f770d3c294283d8aaf9e128ffd4443b56d68cffbdca339f8aa94b11


We can now see the list of our pods.

In [4]:
!podman pod ls

POD ID        NAME        STATUS      CREATED         INFRA ID      # OF CONTAINERS
f0504fbc7f77  my-pod      Created     37 seconds ago  3ddff088879f  1


We can see the list of containers and what pod they are in by running the following command:

In [6]:
!podman ps -a --pod

CONTAINER ID  IMAGE                                    COMMAND     CREATED             STATUS             PORTS       NAMES               POD ID        PODNAME
3ddff088879f  localhost/podman-pause:4.8.1-1701777650              2 minutes ago       Up About a minute              f0504fbc7f77-infra  f0504fbc7f77  my-pod
b867a7c131c1  docker.io/library/alpine:latest          top         About a minute ago  Up About a minute              blissful_feistel    f0504fbc7f77  my-pod


As you can see there is already a container within our pod. This is the infra container, an infra container is a lightweight container used to coordinate the shared kernel namespace of a pod. This container is created by default, you can overwrite this behaviour but it's not a recommended behaviour.

We will run a wordpress workload within our pod, we will need to run a mariadb container as backend and a wordpress container as frontend. Let's first deploy the database:

In [10]:
!podman run -d --pod my-pod \
  -e MYSQL_ROOT_PASSWORD="hpedev" \
  -e MYSQL_DATABASE="wp" \
  -e MYSQL_USER="wordpress" \
  -e MYSQL_PASSWORD="wordpress" \
  --name=wp-db \
  docker.io/library/mariadb:latest

Trying to pull docker.io/library/mariadb:latest...
Getting image source signatures
Copying blob 3dd181f9be59 [-------------] 0.0b / 29.0MiB (skipped: 0.0b = 0.00%)
Copying blob 3dd181f9be59 [----------------------] 37.3KiB / 29.0MiB | 5.3 MiB/s
Copying blob 3b35d6b19e4c [-------------] 0.0b / 83.4MiB (skipped: 0.0b = 0.00%)
Copying blob 3b35d6b19e4c [---------------------------] 0.0b / 83.4MiB | 0.0 b/s
Copying blob 304a7e51ee09 [--------------] 0.0b / 1.7KiB (skipped: 0.0b = 0.00%)
Copying blob 304a7e51ee09 [----------------------------] 0.0b / 1.7KiB | 0.0 b/s
Copying blob 0b9aaebc7c63 [--------------] 0.0b / 332.0b (skipped: 0.0b = 0.00%)
Copying blob 0b9aaebc7c63 [----------------------------] 0.0b / 332.0b | 0.0 b/s
Copying blob 72b8a6ebd591 [--------------] 0.0b / 5.4MiB (skipped: 0.0b = 0.00%)
Copying blob 72b8a6ebd591 [----------------------------] 0.0b / 5.4MiB | 0.0 b/s
Copying blob 56cb545d2ee4 [--------------] 0.0b / 147.0b (skipped: 0.0b = 0.00%)
Copying blob 56cb545d2ee4 

There are a couple of important points to mention regarding previous command. First, we have specified that our container should be part of our pod with the option "--pod". Second we are passing environment variables to the container with the option "-e", by doing this we can do some basic configuration for this instance of the container.

If we list the containers and pods again we should see two containers in the same pod.

In [11]:
!podman ps -a --pod

CONTAINER ID  IMAGE                                    COMMAND     CREATED         STATUS                           PORTS       NAMES               POD ID        PODNAME
3ddff088879f  localhost/podman-pause:4.8.1-1701777650              16 minutes ago  Up 13 seconds                                f0504fbc7f77-infra  f0504fbc7f77  my-pod
b867a7c131c1  docker.io/library/alpine:latest          top         14 minutes ago  Exited (143) About a minute ago              blissful_feistel    f0504fbc7f77  my-pod
fb072a7e5a6c  docker.io/library/mariadb:latest         mariadbd    12 seconds ago  Up 13 seconds                                wordpress-test-db   f0504fbc7f77  my-pod


We can also use the following command:

In [8]:
!podman pod ps

POD ID        NAME        STATUS      CREATED        INFRA ID      # OF CONTAINERS
f0504fbc7f77  my-pod      Running     5 minutes ago  3ddff088879f  2


Let's add the second container, the fronend.

In [14]:
!podman run -d  --pod=my-pod \
  -e WORDPRESS_DB_NAME="wp" \
  -e WORDPRESS_DB_USER="wordpress" \
  -e WORDPRESS_DB_PASSWORD="wordpress" \
  -e WORDPRESS_DB_HOST="127.0.0.1" \
  --name wp-web \
  docker.io/library/wordpress:latest

Trying to pull docker.io/library/wordpress:latest...
Getting image source signatures
Copying blob af107e978371 [-------------] 0.0b / 27.8MiB (skipped: 0.0b = 0.00%)
Copying blob af107e978371 [---------------------------] 0.0b / 27.8MiB | 0.0 b/s
Copying blob 74f0c50b3097 [--------------] 0.0b / 485.0b (skipped: 0.0b = 0.00%)
Copying blob 74f0c50b3097 [----------------------------] 0.0b / 485.0b | 0.0 b/s
Copying blob 6480d4ad61d2 [--------------] 0.0b / 227.0b (skipped: 0.0b = 0.00%)
Copying blob 6480d4ad61d2 [----------------------------] 0.0b / 227.0b | 0.0 b/s
Copying blob 673e01769ec9 [-------------] 0.0b / 19.4MiB (skipped: 0.0b = 0.00%)
Copying blob 673e01769ec9 [---------------------------] 0.0b / 19.4MiB | 0.0 b/s
Copying blob 0ebe7ec824ca [--------------] 0.0b / 272.0b (skipped: 0.0b = 0.00%)
Copying blob 0ebe7ec824ca [----------------------------] 0.0b / 272.0b | 0.0 b/s
Copying blob 95f5176ece8b [-------------] 0.0b / 99.5MiB (skipped: 0.0b = 0.00%)
Copying blob 95f5176ece8

As you can see we've used the address 127.0.0.1 for the environment variable WORDPRESS_DB_HOST, this means we're pointing to localhost for finding the database. This is only possible because both containers (the web and the database) are running in the same pod.

Checking the logs we can see there is no issue with the database, confirming it works properly:

In [16]:
!podman logs wp-web

WordPress not found in /var/www/html - copying now...
Complete! WordPress has been successfully copied to /var/www/html
No 'wp-config.php' found in /var/www/html, but 'WORDPRESS_...' variables supplied; copying 'wp-config-docker.php' (WORDPRESS_DB_HOST WORDPRESS_DB_NAME WORDPRESS_DB_PASSWORD WORDPRESS_DB_USER)
AH00558: apache2: Could not reliably determine the server's fully qualified domain name, using 10.0.2.100. Set the 'ServerName' directive globally to suppress this message
AH00558: apache2: Could not reliably determine the server's fully qualified domain name, using 10.0.2.100. Set the 'ServerName' directive globally to suppress this message
[Thu Dec 21 12:29:45.926663 2023] [mpm_prefork:notice] [pid 1] AH00163: Apache/2.4.57 (Debian) PHP/8.2.13 configured -- resuming normal operations
[Thu Dec 21 12:29:45.926691 2023] [core:notice] [pid 1] AH00094: Command line: 'apache2 -D FOREGROUND'


Last, I'd like to mention that it's possible to create a systemd unit file for a pod. You could do that the same way we did it for a container in previous section of this workshop.